# Import aller nötigen Module

In [57]:
import os

import numpy as np
import pandas as pd
#Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
# Filtering
from scipy.signal import butter, filtfilt
from scipy import stats

# Hilfsfunktionen
Hier definieren wir unsere Hilfsfunktionen zum Plotten

In [58]:
#Vordefinierte Hilfsfunktion zum plotten
def plot_filter(dataframe, data_column_name, filtered_column_name, title):
    fig = px.line(dataframe, x='Time (s)', y=[data_column_name, filtered_column_name], title=title)
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text=filtered_column_name)
    fig.show() 

# Datenimport

Hier laden wir unsere .pyphox Dateien und fangen ab, falls diese nicht vorhanden sein solte. Zudem setzen wir schoonmal die Spaltennamen fest um diese im fortlaufenen Programm nutzen zu können

In [60]:
file_name = "RawDataWisch.csv"

# Error-Handling
try:
    dataframe = pd.read_csv("RawDataWisch.csv")
except FileNotFoundError:
    print(f"Error: File not found")
    exit()


# Spaltennamen festsetzen: 
time_col = "Time (s)"
gx_col = "Linear Acceleration x (m/s^2)"
gy_col = "Linear Acceleration y (m/s^2)"
gz_col = "Linear Acceleration z (m/s^2)"
abs_col = "Absolute acceleration (m/s^2)"

# Preview of Data
print("Dataset:")
print(dataframe.head())

Dataset:
   Time (s)  Linear Acceleration x (m/s^2)  Linear Acceleration y (m/s^2)  \
0  0.093435                          0.003                          0.139   
1  0.098417                          0.003                          0.143   
2  0.103398                          0.003                          0.129   
3  0.108380                          0.003                          0.115   
4  0.113361                          0.003                          0.098   

   Linear Acceleration z (m/s^2)  Absolute acceleration (m/s^2)  
0                         -0.007                       0.139208  
1                         -0.008                       0.143255  
2                         -0.007                       0.129225  
3                         -0.006                       0.115195  
4                         -0.006                       0.098229  


# Frequenzfilter

Hier wird das Rauschen reduziert. Dafür nutzen wir Butterworth aus der Vorlesung. Damit werden hoch- bzw. niederfrequente Signale gefiltert. 
Zuerst folgen die niederfrequenten Signale, dann die hochfrequenten:

In [61]:
filtered_dataframe = dataframe.copy()

filter_order = 4  

low_cutoff = 0.5
high_cutoff = 5 

# Hier nutzen wir den butterwort filter um die Daten zu glätten
b, a = butter(filter_order, [low_cutoff, high_cutoff], btype='band', analog=False, fs =200)

# Apply the filter to acceleration signals
filtered_dataframe['filtered_X_band'] = filtfilt(b, a, dataframe['Linear Acceleration x (m/s^2)'])
filtered_dataframe['filtered_Y_band'] = filtfilt(b, a, dataframe['Linear Acceleration y (m/s^2)'])
filtered_dataframe['filtered_Z_band'] = filtfilt(b, a, dataframe['Linear Acceleration z (m/s^2)'])

plot_filter(filtered_dataframe, 'Linear Acceleration x (m/s^2)', 'filtered_X_band', 'Acceleration X with Band-Pass Filtering')
plot_filter(filtered_dataframe, 'Linear Acceleration y (m/s^2)', 'filtered_Y_band', 'Acceleration Y with Band-Pass Filtering')
plot_filter(filtered_dataframe, 'Linear Acceleration z (m/s^2)', 'filtered_Z_band', 'Acceleration Z with Band-Pass Filtering')

# Ausreißer Bereinigung
Hier wollen wir nun die möglichen Ausreißer bereinigen: 

In [62]:
# Berechnung der Z-Scores für jede Spalte (X, Y, Z)
filtered_dataframe['Z_Score_X'] = stats.zscore(filtered_dataframe['Linear Acceleration x (m/s^2)'])
filtered_dataframe['Z_Score_Y'] = stats.zscore(filtered_dataframe['Linear Acceleration y (m/s^2)'])
filtered_dataframe['Z_Score_Z'] = stats.zscore(filtered_dataframe['Linear Acceleration z (m/s^2)'])

# Festlegen eines Schwellenwerts für Z-Scores, ab dem ein Punkt als Ausreißer betrachtet wird
z_score_threshold = 4  

# Filtern der Ausreißer aus dem filterd_dataframe (nur wenn alle drei Achsen unterhalb des Threshold liegen sind Daten ok)
filtered_dataframe = filtered_dataframe[(abs(filtered_dataframe['Z_Score_X']) <= z_score_threshold) &
                     (abs(filtered_dataframe['Z_Score_Y']) <= z_score_threshold) &
                     (abs(filtered_dataframe['Z_Score_Z']) <= z_score_threshold)]



# Jetzt enthält 'filtered_dataframe' nur noch die Datenpunkte, die nicht als Ausreißer betrachtet werden.
print(dataframe.shape)
print(filtered_dataframe.shape)

#Plotting
plot_filter(filtered_dataframe, 'Linear Acceleration x (m/s^2)', 'Z_Score_X', 'Acceleration X with Z-Score Filtering')
plot_filter(filtered_dataframe, 'Linear Acceleration y (m/s^2)', 'Z_Score_Y', 'Acceleration Y with Z-Score Filtering')
plot_filter(filtered_dataframe, 'Linear Acceleration z (m/s^2)', 'Z_Score_Z', 'Acceleration Z with Z-Score Filtering')


(3998, 5)
(3993, 11)


# Windowing
Hier nutzen wir Windowing um die Daten zu zerlegen. Das brauchen wir später für die Features-Berechnung

In [63]:
# Da fs = 200 Hz setzen wir das Fenster auf 0.5 Sekunden um auch kurze Bewegungsereignisse abzubilden. 
# Wir könnten window_size aber auch auf 200 setzen.
window_size = 100 
overlap = 50 

#Liste initiieren für Fenster
windows = []

#Für jedes Fenster in den Daten neuen Dataframe erstellen, der in Liste abgelegt wird 
for i in range(0, len(filtered_dataframe), window_size - overlap):
    window = filtered_dataframe.iloc[i:i+window_size]
    windows.append(window)

#Ersten Dataframe in der Liste ausgeben lassen 
print(windows[0].head())
#Anzahl Datenpunkte des ersten Dataframes ausgeben lassen 
print(len(windows[0]))
#Spalten des ersten Dataframes ausgeben lassen 
print(windows[0].columns)

#Ausgeben lassen wie viele Dataframes also Fenster entstanden sind 
print (len(windows))


   Time (s)  Linear Acceleration x (m/s^2)  Linear Acceleration y (m/s^2)  \
0  0.093435                          0.003                          0.139   
1  0.098417                          0.003                          0.143   
2  0.103398                          0.003                          0.129   
3  0.108380                          0.003                          0.115   
4  0.113361                          0.003                          0.098   

   Linear Acceleration z (m/s^2)  Absolute acceleration (m/s^2)  \
0                         -0.007                       0.139208   
1                         -0.008                       0.143255   
2                         -0.007                       0.129225   
3                         -0.006                       0.115195   
4                         -0.006                       0.098229   

   filtered_X_band  filtered_Y_band  filtered_Z_band  Z_Score_X  Z_Score_Y  \
0        -2.567775         1.750328        -0.116625   0

# Features erstellen
Hier berechnen wir die Merkmale, die wesentliche Eigneschaften der Daten kompakt beschreiben. Diese dienen als Eingabe für den späteren Entscheidungsbaum. 

In [ ]:
#Leere Liste erstellen, um Features zu speichern 
feature_list = []

#Jeden Dataframe in Windows einzeln durchgehen und für jede der drei Achsen die Features berechnen 
for window in windows:
    acceleration_data = window[['Linear Acceleration x (m/s^2)', 'Linear Acceleration y (m/s^2)', 'Linear Acceleration z (m/s^2)']]
    #Mittelwert
    acceleration_mean = acceleration_data.mean()
    #Standardabweichung
    acceleration_std_dev = acceleration_data.std()
    #Minimum
    acceleration_min = acceleration_data.min()
    #Maximum
    acceleration_max = acceleration_data.max()
    
    #Berechnete Werte an Liste anhängen, unter entsprechend benannten Spalten 
    feature_list.append({
        'BeschleunigungX_Mittelwert': acceleration_mean.iloc[0],
        'BeschleunigungY_Mittelwert': acceleration_mean.iloc[1],
        'BeschleunigungZ_Mittelwert': acceleration_mean.iloc[2],
        'BeschleunigungX_Standardabweichung': acceleration_std_dev.iloc[0],
        'BeschleunigungY_Standardabweichung': acceleration_std_dev.iloc[1],
        'BeschleunigungZ_Standardabweichung': acceleration_std_dev.iloc[2],
        'BeschleunigungX_Minimum': acceleration_min.iloc[0],
        'BeschleunigungY_Minimum': acceleration_min.iloc[1],
        'BeschleunigungZ_Minimum': acceleration_min.iloc[2],
        'BeschleunigungX_Maximum': acceleration_max.iloc[0],
        'BeschleunigungY_Maximum': acceleration_max.iloc[1],
        'BeschleunigungZ_Maximum': acceleration_max.iloc[2]
    })

#In einen Pandas Dataframe konvertieren und ausgeben
feature_df = pd.DataFrame(feature_list)

print(feature_df.head())

#Dimensionen der Feautures ausgeben lassen 
feature_df.shape


   BeschleunigungX_Mittelwert  BeschleunigungY_Mittelwert  \
0                    -1.21583                     0.31469   
1                     5.21566                    -4.78280   
2                     8.73415                    -7.42123   
3                     0.09525                    -7.77677   
4                   -11.38799                    -3.15145   

   BeschleunigungZ_Mittelwert  BeschleunigungX_Standardabweichung  \
0                    -0.36127                            3.206064   
1                    -0.33389                            7.353071   
2                     2.06814                            4.514616   
3                     2.54067                            7.852754   
4                    -0.91286                            7.731296   

   BeschleunigungY_Standardabweichung  BeschleunigungZ_Standardabweichung  \
0                            2.397561                            1.716349   
1                            4.742098                           

C:\Users\denni\AppData\Local\Temp\ipykernel_3908\3505432228.py:18: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\denni\AppData\Local\Temp\ipykernel_3908\3505432228.py:19: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\denni\AppData\Local\Temp\ipykernel_3908\3505432228.py:20: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\denni\AppData\Local\Temp\ipykernel_3908\3505432228.py:21: FutureWarning:

Series.__getit

(80, 12)